### Visualise results of 1D nodes of a threedimodel


Load libraries

In [ ]:
import requests
from pathlib import Path
from getpass import getpass

import numpy as np
import pandas as pd

from pyproj import Transformer

from threedigrid.admin.gridresultadmin import GridH5ResultAdmin
from threedigrid.admin.gridadmin import GridH5Admin
from threedi_api_client.api import ThreediApi
from threedi_api_client.openapi.api.v3_api import V3Api
from threedi_api_client.files import download_file

from ipyleaflet import Map, basemaps, CircleMarker, LegendControl
from branca.colormap import linear

Authentication

In [ ]:
# Provide authentication details
API_HOST = "https://api.3di.live"
PERSONAL_API_KEY = getpass("Personal API token")  # https://management.3di.live/personal_api_keys

config = {
    "THREEDI_API_HOST": API_HOST,
    "THREEDI_API_PERSONAL_API_TOKEN": PERSONAL_API_KEY
}

api_client: V3Api = ThreediApi(config=config)

Load model and print its properties

In [ ]:
MODELNAME = input("Input modelname for filtering simulations")

threedi_model = api_client.threedimodels_list(name__icontains=MODELNAME).results[0]
print(threedi_model)

Get the results from simulation

In [ ]:
simulation_results = api_client.simulations_list(name__contains=MODELNAME)

if simulation_results.count == 0:
    raise Exception(f"No simulations found for model {MODELNAME}")

simulation = simulation_results.results[0]

status = api_client.simulations_status_list(simulation.id)

print(simulation)
print(f"status: {status}")
assert status.name == 'finished'

Download the result files from the threedimodel

In [ ]:
# get result files
result_files = api_client.simulations_results_files_list(simulation.id)

for result in result_files.results:
    print(result)

# download result files
download_folder = Path(f'results_{simulation.id}')
download_folder.mkdir(exist_ok=True)

for file in result_files.results:
    download = api_client.simulations_results_files_download(
        id=file.id, simulation_pk=simulation.id
    )
    file_path = download_folder / file.filename
    download_file(download.get_url, file_path)
    print(f"Finished downloading {file.filename}")

# download grid administration file
threedi_model_id = simulation.threedimodel_id
download_url = api_client.threedimodels_gridadmin_download(threedi_model_id)
file_path = download_folder / "gridadmin.h5"
r = requests.get(download_url.get_url, timeout=600)
with open(file_path, "wb") as f:
    for chunk in r.iter_content(chunk_size=8192):
        f.write(chunk)
print(f"Finished downloading gridadmin.h5")

Create dataframe with coordindates and waterlevels

In [ ]:
# define paths and files
f = download_folder / "gridadmin.h5"
nc = download_folder / "results_3di.nc"

# load files into gr object
ga = GridH5Admin(str(f))
gr = GridH5ResultAdmin(str(f), str(nc))

# set chunksize
gr.set_timeseries_chunk_size(200)

# create pandas df with coordinates and water level of last timestep
node_df = pd.DataFrame(
    {
        "x": ga.nodes.coordinates[0],
        "y": ga.nodes.coordinates[1],
        "s1": gr.nodes.data["s1"][-1],
        "lat": np.nan,
        "lon": np.nan,
    })

# # convert x and y in EPSG 28992 to lat and lon
transformer = Transformer.from_crs("epsg:28992", "epsg:4326")
for i in range(len(node_df)):
    node_df.loc[i, 'lat'] = transformer.transform(node_df.x[i], node_df.y[i])[0]
    node_df.loc[i, 'lon'] = transformer.transform(node_df.x[i], node_df.y[i])[1]

Plot the waterlevels on a map between min and maximum

In [ ]:
# get extent of model
extent_coordinates = threedi_model.extent_one_d.coordinates

# get center and create map
center = (extent_coordinates[0][1] + extent_coordinates[1][1]) / 2.0, (extent_coordinates[0][0] + extent_coordinates[1][0]) / 2.0
m = Map(center=center, zoom=10, basemap=basemaps.Esri.WorldImagery)

# set min and max water level for colormap
MIN = input("Set minimum water level for colormap")
MAX = input("Set maximum water level for colormap")

# add circle markers to the map
colormap = linear.BuPu_03.scale(float(MIN), float(MAX))
for i in node_df.lat.index:
    marker = CircleMarker(
        location=[node_df.lat[i], node_df.lon[i]],
        radius=5,
        color=colormap(node_df.s1[i]),
        fill=True,
        fill_color=colormap(node_df.s1[i]),
        fill_opacity=0.8,
        weight=0)
    m.add_layer(marker)

# add the colormap as a legend to the map
legend = LegendControl({MAX:"#810f7c", MIN:"#edf8fb"}, name="waterlevel", position="topright")
m.add_control(legend)

# show the map
m